FastAPI + Docker + Streamlit

reference: https://www.analyticsvidhya.com/blog/2023/02/build-and-deploy-an-ml-model-app-using-streamlit-docker-and-gke/

### Introduction

You have a dataset, did some data analysis, and built a model around it; now, what? The next step will be to deploy the model on a server, so your model will be accessible to the general public or your development team to integrate it with the app. 

So, in this notebook, you will learn how to

- Serve a machine learning model for predicting employee churn as a Web service using Fast API.
- Create a simple web front end using Streamlit.
- Dockerizing the Streamlit app and API.

### Model Building

Import libraries

In [17]:
import pandas as pd 
from sklearn.preprocessing import LabelEncoder

Load the data

In [18]:
df = pd.read_csv('example4_dc.csv')
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Departments,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


Prepare the data

In [19]:
#encode categorical data
enc = LabelEncoder()
df['Departments'] = enc.fit_transform(df['Departments'])
df['salary'] = enc.fit_transform(df['salary'])

Split training-test

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
y = df['left']
df.drop('left', axis=1, inplace=True)
x_train, x_test, y_train, y_test = train_test_split(df, y, test_size=0.15)

Import libraries for model building

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Create a custom switcher class

In [23]:
class my_classifier(BaseEstimator,):
    def __init__(self, estimator=None):
        self.estimator = estimator
    def fit(self, X, y=None):
        self.estimator.fit(X,y)
        return self
    def predict(self, X, y=None):
        return self.estimator.predict(X,y)
    def predict_proba(self, X):
        return self.estimator.predict_proba(X)
    def score(self, X, y):
        return self.estimator.score(X, y)

Create a pipeline and pass parameters. We will be using a Random Forest classifier with multiple hyperparameters.

In [24]:
pipe = Pipeline([ ('clf', my_classifier())])
parameters = [
             {'clf':[RandomForestClassifier()],
             'clf__n_estimators': [75, 100, 125,],
             'clf__min_samples_split': [2,4,6],
             'clf__max_depth': [5, 10, 15,]
             },
           ]

Create a GridsearchCV object and fit the model with it

In [ ]:
grid = GridSearchCV(pipe, parameters, cv=5, scoring='roc_auc')
grid.fit(x_train,y_train)
#
model = grid.best_estimator_
score = grid.best_score_

Use pickle to serialize the model

In [26]:
import pickle

In [ ]:
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

### Create a Rest API

The next step is to wrap our model with a Rest API. This allows us to access our saved model as and when required. We can get our prediction via an HTTP request to an API endpoint. For this, we will be using Fast API. 

import libraries

In [29]:
from fastapi import FastAPI
from pydantic import BaseModel
from joblib import load
import pandas as pd
import json

Instantiate the FastAPI class and load the model

In [ ]:
app = FastAPI()
model  = load('model.pkl')

Build a pydantic model

In [ ]:
class user_input(BaseModel):
    satisfaction_level  : float
    last_evaluation     : float
    number_project      : int
    average_montly_hours: int
    time_spend_company  : int
    Work_accident       : int  
    promotion_last_5years: int
    departments          : str
    salary              : str

Create a prediction class to convert data in the appropriate format for predicting

In [ ]:
def predict(data):
    departments_list = ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical']
    data[-2] = departments_list.index(data[-2])
    salaries = ['low', 'medium', 'high']
    data[-1] = salaries.index(data[-1])
    columns = ['satisfaction_level', 'last_evaluation', 
                'number_project', 'average_montly_hours', 'time_spend_company', 
                'Work_accident', 'promotion_last_5years','departments', 'salary']
    prediction = model.predict( pd.DataFrame([data], columns= columns))
    proba = model.predict_proba(pd.DataFrame([data], columns= columns))
    return prediction, proba

Create the Base Endpoint

In [ ]:
@app.get('/')
def welcome():
    return f'Welcome to our app api'

Create an endpoint for prediction

In [ ]:
@app.post('/predict')
def func(Input:user_input):
    data = [Input.satisfaction_level, Input.last_evaluation, 
            Input.number_project, Input.average_montly_hours, 
            Input.time_spend_company, Input.Work_accident, 
            Input.promotion_last_5years, Input.departments, Input.salary]
    pred, proba = predict(data)
    output = {'prediction':int(pred[0]), 'probability':float(proba[0][1])}
    return json.dumps(output)

To view the API, run the below script on your terminal

```bash
unicorn main:app --reload
```
```

### Streamlit app

We will create a new file called streamlit-app.py and add the below code to it.

Import libraries

In [ ]:
import streamlit as st
import requests
import json

Define the headers

In [ ]:
st.title('End-to-End App') #title to be shown
st.header('Enter the employee data:') #header to be shown in app

Create input forms

In [ ]:
satisfaction_level = st.number_input('satisfaction level',min_value=0.00, max_value=1.00)
last_evaluation = st.number_input('last evaluation score',min_value=0.00, max_value=1.00)
number_project = st.number_input('number of projects',min_value=1)
average_montly_hours = st.slider('average monthly hours', min_value=0, max_value=320)
time_spend_company = st.number_input(label = 'Number of years at company', min_value=0)
Work_accident = st.selectbox('If met an accident at work', [1,0], index = 1)
promotion_last_5years = st.selectbox('Promotion in last 5 years yes=1/no=0', [1,0], index=1)
departments = st.selectbox('Department', ['IT', 'RandD', 'accounting', 'hr', 'management', 'marketing', 'product_mng', 'sales', 'support', 'technical'])
salary = st.selectbox('Salary Band', ['low', 'medium', 'high',])

Create a dictionary of the above variables with keys.

In [ ]:
names = ['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'departments', 'salary']
params = [satisfaction_level, last_evaluation, number_project,
       average_montly_hours, time_spend_company, Work_accident,
       promotion_last_5years, departments, salary]
input_data = dict(zip(names, params))

Predict the output

In [ ]:
if st.button('Predict'):
    #pred = predict(satisfaction_level, last_evaluation, number_project, average_montly_hours, time_spend_company, 
    #                                         Work_accident, promotion_last_5years,department, salary)
    try:
        output_ = requests.post(url = 'http://localhost:8000/predict', data = json.dumps(input_data))
    except:
       print('Not able to connect to api server')
    #output_ = requests.post(url = 'http://localhost:8000/predict', data = json.dumps(input_data))
    ans = eval(output_.json())
    output = 'Yes' if ans['prediction']==1 else 'No'
    if output == 'Yes':
        st.success(f"The employee might leave the company with a probability of {(ans['probability'])*100: .2f}")
    if output == 'No':
        st.success(f"The employee might not leave the company with a probability of {(1-ans['probability'])*100: .2f}")

To launch the app, type the code bellow

```bash
streamlit run streamlit-app.py
```

### Containerzing the App

To Dockerize the apps, we first need to create a docker file for each component in their respective directory.

Dockerfile for Rest API

```Dockerfile
FROM python:3.9

COPY requirements.txt app/requirements.txt

WORKDIR /app

RUN pip install -r requirements.txt

COPY . /app

EXPOSE 8000

CMD ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", "8000" , "--reload"]

```

To elaborate on the last command in the backend Dockerfile, the following are the defined settings for Uvicorn:

— host 0.0.0.0 defines the address to host the server on.

— port 8008 defines the port to host the server on.

main:app tells Uvicorn where it can find the FastAPI ASGI application — e.g., “within the the ‘main.py’ file, you’ll find the ASGI app, app = FastAPI().

— reload enables auto-reload so the server will restart after changes are made to the code base.

Remember to add the requirements.txt file in the same directory as the Dockerfile. The requirements.txt file contains all the libraries that are required to run the API.

```bash
pip freeze > requirements.txt
```
```

Create a docker file for the Streamlit app

```Dockerfile

# frontent/Dockerfile

FROM python:3.9

COPY requirements.txt app/requirements.txt

WORKDIR /app

RUN pip install -r requirements.txt

COPY . /app

EXPOSE 8501

ENTRYPOINT ["streamlit","run"]
CMD ["app.py"]
    
``` 

### Docker Composer

Normaly we would have to build each docker file separately and run them individually. But with docker composer, we can run all the docker files at once!!

Docker Compose is used when we have a seperate docker file for each component of the app. In our case, we have two docker files, one for the API(backend) and the other for the Streamlit app(frontend).

We define a docker-compose.yml file in the root directory of the project. The docker-compose.yml file contains all the information required to run the app. 

```yml
version: "2"
services:
  app:
    build: ./frontend
    ports: 
      - '8501:8501'
  main:
    build: ./backend
    ports:
      - '8000:8000'
```

This is our project structure
```bash
├── docker-compose.yml
├── backend
├── misc
└── frontend
```

Then to run our docker composer

```bash
docker-compose up -d --build
```

Two containers will be running!

Next time you dont need to rebuid the images

```bash
docker-compose up -d
```